**! 사용시 주의 사항.**

훈련시 train, valid가 모두 필요하므로 사전에 데이터를 train, valid 폴더에 수동으로 파일을 나눠서 저장 필요. ex) train: 66255개, valid: 10639개

## 1. 작업 환경  
Ubuntu 18.04.6 LTS  
Cuda 11.2, V11.2.152

## 2. 라이브러리 설정

In [6]:
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 --extra-index-url https://download.pytorch.org/whl/cu110

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu110


## 3. 필요한 파일 다운로드

In [7]:
!git clone https://github.com/bcmin1018/dacon-handwritten-recognition.git

Cloning into 'dacon-handwritten-recognition'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 60 (delta 10), reused 34 (delta 1), pack-reused 0
Unpacking objects: 100% (60/60), 3.26 MiB | 11.44 MiB/s, done.


In [8]:
%cd '/content/dacon-handwritten-recognition/'

/content/dacon-handwritten-recognition


In [9]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 4. 훈련 데이터 생성

In [11]:
%cd '/content/dacon-handwritten-recognition/TRDG2DTRB'

/content/dacon-handwritten-recognition/TRDG2DTRB


In [12]:
ORG_DATA_DIR = '/content/drive/MyDrive/project/Kyowon/data'  # 데이터 경로
ORG_TRAIN_DATA = ORG_DATA_DIR + '/train' # 훈련 데이터 경로
ORG_VALID_DATA = ORG_DATA_DIR + '/valid' # 검증 데이터 경로

CONVERT_DATA_DIR = '/content/dacon-handwritten-recognition/data/convert' # 변환된 데이터 저장 경로
CONVERT_TRAIN_DATA = CONVERT_DATA_DIR + '/train' # 변환된 훈련 데이터 저장 경로
CONVERT_VALID_DATA = CONVERT_DATA_DIR + '/valid' # 변환된 검증 데이터 저장 경로

In [13]:
import convert

이미지 데이터를 모델 input에 맞도록 convert 전에 train, valid 폴더에 수동으로 파일을 나눠서 저장합니다.

In [ ]:
# 아미지 파일이 많은 경우 코랩에서는 시간초과로 input/output 에러가 자주 발생. (여러번 반복하면 성공)
# 코랩에서는 속도가 느림.
convert.run(ORG_TRAIN_DATA, CONVERT_TRAIN_DATA)

input path:  /content/drive/MyDrive/project/Kyowon/data/train
output path:  /content/dacon-handwritten-recognition/data/convert/train

Output folder already exists.

So, delete all data of output folder [/content/dacon-handwritten-recognition/data/convert/train]

Total file count:  10000
 6800 / 10000 Processing !!

In [14]:
convert.run(ORG_VALID_DATA, CONVERT_VALID_DATA)

input path:  /content/drive/MyDrive/project/Kyowon/data/valid
output path:  /content/dacon-handwritten-recognition/data/convert/valid
Total file count:  10000
10000 / 10000 Processing !!
Conversion complete!



In [ ]:
%cd '/content/dacon-handwritten-recognition/deep-text-recognition-benchmark'

In [ ]:
lmdb_path = './result' # lmdb 저장 경로

In [ ]:
import create_lmdb_dataset

In [ ]:
create_lmdb_dataset.createDataset(CONVERT_TRAIN_DATA, CONVERT_TRAIN_DATA+'/gt.txt',lmdb_path+'/train' )

In [ ]:
create_lmdb_dataset.createDataset(CONVERT_VALID_DATA, CONVERT_VALID_DATA+'/gt.txt',lmdb_path+'/valid' )

## 4. 손글씨 인식 사전학습 모델 다운로드
다운로드 링크: https://drive.google.com/file/d/12gyTwu5tM_9MthsXI9Ja2yCOXpvDEIao/view  
사전학습 모델 논문 링크: https://scienceon.kisti.re.kr/srch/selectPORSrchArticle.do?cn=NPAP13451726&dbt=NPAP  
깃헙 링크: https://github.com/parksunwoo/ocr_kor

In [ ]:
!gdown --id 12gyTwu5tM_9MthsXI9Ja2yCOXpvDEIao --output best_accuracy.pth

In [ ]:
%cd '/content/dacon-handwritten-recognition/deep-text-recognition-benchmark'


# 환경에 맞게 배치사이즈 조정 필요
# 테스크에서 성능이 가장 높게 나온 파라미터
!python3 train.py \
	--train_data ./result/train \
	--valid_data ./result/valid \
	--select_data / \
	--batch_ratio 1 \
	--batch_size 70 \
	--Transformation TPS \
	--FeatureExtraction ResNet \
	--SequenceModeling BiLSTM \
	--Prediction Attn \
	--imgH 64 \
	--imgW 224 \
	--data_filtering_off \
	--workers 0 \
	--batch_max_length 50 \
	--saved_model ./best_accuracy.pth \
	--PAD \
	--FT

## 6. test 파일 예측

In [ ]:
# test 파일의 경로를 입력
ORG_TEST_DATA = ORG_DATA_DIR + '/test'

In [ ]:
import pandas as pd
submit = pd.read_csv('../sample_submission.csv')

In [ ]:
submit['label']

0        가
1        가
2        가
3        가
4        가
        ..
74116    가
74117    가
74118    가
74119    가
74120    가
Name: label, Length: 74121, dtype: object

In [ ]:
%cd '/content/dacon-handwritten-recognition/deep-text-recognition-benchmark'
!python3 demo.py \
	--image_folder '/content/drive/MyDrive/project/Kyowon/data/test'  \
	--Transformation TPS \
	--FeatureExtraction ResNet \
	--SequenceModeling BiLSTM \
	--Prediction Attn \
	--imgH 64 \
	--imgW 224 \
	--PAD \
	--saved_model './saved_models/TPS-ResNet-BiLSTM-Attn-Seed1111/best_accuracy.pth' # 훈련에 입력한 파라미터에 따라 폴더 경로가 달리짐.

/content/dacon-handwritten-recognition/deep-text-recognition-benchmark
model input parameters 64 224 20 1 512 256 974 25 TPS ResNet BiLSTM Attn
loading pretrained model from ./saved_models/TPS-ResNet-BiLSTM-Attn-Seed1111/best_accuracy.pth
123123123
--------------------------------------------------------------------------------
image_path               	predicted_labels         	confidence score
--------------------------------------------------------------------------------
/content/drive/MyDrive/project/Kyowon/data/test/TEST_00000.png	[GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO
/content/drive/MyDrive/project/Kyowon/data/test/TEST_00001.png	[GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO
/content/drive/MyDrive/project/Kyowon/data/test/TEST_00002.png	[GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO
/content/drive/MyDr

참고 사이트
1. https://davelogs.tistory.com/77
2. https://github.com/DaveLogs/TRDG2DTRB.git
3. https://github.com/clovaai/deep-text-recognition-benchmark
